In [1]:
import pandas as pd
import subprocess

In [2]:
TARGET_COLUMNS = [ 
    'hospital_ccn',
    'hospital_ein',
    'code_type',
    'internal_code',
    'description',
    'code_orig',
    'code_prefix',
    'code',
    'modifier',
    'ndc',
    'ndc_num_hcpcs_doses',
    'ndc_quantity_desc',
    'apc',
    'rev_code',
    'rev_desc',
    'billing_class',
    'patient_class',
    'payer_category',
    'payer_orig',
    'payer_name',
    'plan_orig',
    'plan_name',
    'plan_id',
    'plan_type',
    'rate',
    'rate_method',
    'rate_desc',
    'filename',
    'file_last_updated',
    'url',
    'permalink'
]

In [3]:
ccn = "450774"
url = "https://www.tops-hospital.com/wp-content/uploads/760246499_TOPSSurgicalSpecialtyHospital_standardcharges.json"
filename = url.split("/")[-1]
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [4]:
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘760246499_TOPSSurgicalSpecialtyHospital_standardcharges.json’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://www.tops-hospital.com/wp-content/uploads/760246499_TOPSSurgicalSpecialtyHospital_standardcharges.json', '-O', '760246499_TOPSSurgicalSpecialtyHospital_standardcharges.json'], returncode=1)

In [5]:
import json

in_f = open(filename, "r")
raw_json_dict = json.load(in_f)
in_f.close()

In [6]:
df = pd.DataFrame()

for json_arr_dict in raw_json_dict.get("data", []):
    if len(json_arr_dict) == 1 and type(json_arr_dict[0]) == dict and json_arr_dict[0].get('file_create_date') is not None:
        row = json_arr_dict[0]
        
        rows = []
        
        if "<and>" in row.get("code"):
            print("Splitting ", row.get("code"))
            codes = row.get("code").split("<and>")
            for code in codes:
                print(code)
                new_row = dict(row)
                new_row['code'] = code
                rows.append(new_row)
        else:
            rows = [row]
        
        for row in rows:
            df = pd.concat([df, pd.DataFrame(row, index=row.keys())], ignore_index=True)

Splitting  72265<and>62284
72265
62284
Splitting  72255<and>62284
72255
62284
Splitting  72265<and>62284
72265
62284
Splitting  24220<and>73085
24220
73085


In [7]:
df = df.drop_duplicates()
df

,file_create_date,run_id,name,tax_id,code,code type,code description,payer,patient_class,gross charge,de-identified minimum negotiated charge,payer-specific negotiated charge,de-identified maximum negotiated charge,discounted cash price,charge_code
0,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,29807,cpt,Repair Of Shoulder Socket Cartilage Using An E...,aetna,O,31355.71,7545.13,7545.13,7911.43,10974.50,NaN
14,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,43235,cpt,"Diagnostic Examination Of Esophagus, Stomach, ...",aetna,O,13684.00,399.79,1625.45,3412.66,4789.40,NaN
28,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,36561,cpt,Insertion Of Central Venous Catheter And Impla...,aetna,O,19250.00,2832.38,2863.73,9101.24,6737.50,NaN
42,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,64492,cpt,Injections Of Upper Or Middle Spine Facet Join...,aetna,O,20764.93,834.32,1837.62,1837.62,7267.73,NaN
56,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,29891,cpt,Removal Of Bone Defect Of Shin And/Or Ankle Us...,aetna,O,60164.85,2824.32,9787.50,34482.47,21057.70,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57464,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,93971,cpt,Venous Doppler- Extremity Unilateral 93971,NaN,NaN,1583.00,NaN,N/A,NaN,NaN,1714174
57474,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,63662,cpt,REMOVAL OF SPINAL NEUROSTIMULATOR ELEC. PLATE/...,NaN,NaN,2991.00,NaN,N/A,NaN,NaN,2580939
57484,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,27497,cpt,FASCIOTOMY W/DEBRID.OF NOVIABLE THIGH OR KNEE ...,NaN,NaN,2991.00,NaN,N/A,NaN,NaN,1480944
57494,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,73630,cpt,XR Foot Complete Left,NaN,NaN,664.00,NaN,N/A,NaN,NaN,630548


In [8]:
df = df.rename(columns={
    'file_create_date': 'file_last_updated',
    'tax_id': 'hospital_ein',
    'code type': 'code_type',
    'code description': 'description',
    'payer': 'payer_name',
    'charge_code': 'internal_code'
})
df

,file_last_updated,run_id,name,hospital_ein,code,code_type,description,payer_name,patient_class,gross charge,de-identified minimum negotiated charge,payer-specific negotiated charge,de-identified maximum negotiated charge,discounted cash price,internal_code
0,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,29807,cpt,Repair Of Shoulder Socket Cartilage Using An E...,aetna,O,31355.71,7545.13,7545.13,7911.43,10974.50,NaN
14,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,43235,cpt,"Diagnostic Examination Of Esophagus, Stomach, ...",aetna,O,13684.00,399.79,1625.45,3412.66,4789.40,NaN
28,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,36561,cpt,Insertion Of Central Venous Catheter And Impla...,aetna,O,19250.00,2832.38,2863.73,9101.24,6737.50,NaN
42,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,64492,cpt,Injections Of Upper Or Middle Spine Facet Join...,aetna,O,20764.93,834.32,1837.62,1837.62,7267.73,NaN
56,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,29891,cpt,Removal Of Bone Defect Of Shin And/Or Ankle Us...,aetna,O,60164.85,2824.32,9787.50,34482.47,21057.70,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57464,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,93971,cpt,Venous Doppler- Extremity Unilateral 93971,NaN,NaN,1583.00,NaN,N/A,NaN,NaN,1714174
57474,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,63662,cpt,REMOVAL OF SPINAL NEUROSTIMULATOR ELEC. PLATE/...,NaN,NaN,2991.00,NaN,N/A,NaN,NaN,2580939
57484,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,27497,cpt,FASCIOTOMY W/DEBRID.OF NOVIABLE THIGH OR KNEE ...,NaN,NaN,2991.00,NaN,N/A,NaN,NaN,1480944
57494,2023-03-15,6145a090-c343-11ed-a585-536ad537c29c,TOPS Surgical Specialty Hospital,76-0246499,73630,cpt,XR Foot Complete Left,NaN,NaN,664.00,NaN,N/A,NaN,NaN,630548


In [9]:
money_columns = df.columns.to_list()[9:-1]
money_columns

['gross charge',
 'de-identified minimum negotiated charge',
 'payer-specific negotiated charge',
 'de-identified maximum negotiated charge',
 'discounted cash price']

In [10]:
remaining_columns = list(set(df.columns.to_list()) - set(money_columns))
df = pd.melt(df, id_vars=remaining_columns, var_name='payer_orig', value_name='rate')
df

,code_type,code,name,run_id,hospital_ein,description,patient_class,payer_name,internal_code,file_last_updated,payer_orig,rate
0,cpt,29807,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Repair Of Shoulder Socket Cartilage Using An E...,O,aetna,NaN,2023-03-15,gross charge,31355.71
1,cpt,43235,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,"Diagnostic Examination Of Esophagus, Stomach, ...",O,aetna,NaN,2023-03-15,gross charge,13684.00
2,cpt,36561,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Insertion Of Central Venous Catheter And Impla...,O,aetna,NaN,2023-03-15,gross charge,19250.00
3,cpt,64492,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Injections Of Upper Or Middle Spine Facet Join...,O,aetna,NaN,2023-03-15,gross charge,20764.93
4,cpt,29891,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Removal Of Bone Defect Of Shin And/Or Ankle Us...,O,aetna,NaN,2023-03-15,gross charge,60164.85
...,...,...,...,...,...,...,...,...,...,...,...,...
26320,cpt,93971,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Venous Doppler- Extremity Unilateral 93971,NaN,NaN,1714174,2023-03-15,discounted cash price,NaN
26321,cpt,63662,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,REMOVAL OF SPINAL NEUROSTIMULATOR ELEC. PLATE/...,NaN,NaN,2580939,2023-03-15,discounted cash price,NaN
26322,cpt,27497,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,FASCIOTOMY W/DEBRID.OF NOVIABLE THIGH OR KNEE ...,NaN,NaN,1480944,2023-03-15,discounted cash price,NaN
26323,cpt,73630,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,XR Foot Complete Left,NaN,NaN,630548,2023-03-15,discounted cash price,NaN


In [11]:
df['payer_category'] = df['payer_orig'].replace('gross charge', 'gross').replace(
    'de-identified minimum negotiated charge','min').replace(
    'de-identified maximum negotiated charge','max').replace(
    'payer-specific negotiated charge', 'payer').replace(
    'discounted cash price', 'cash')
df

,code_type,code,name,run_id,hospital_ein,description,patient_class,payer_name,internal_code,file_last_updated,payer_orig,rate,payer_category
0,cpt,29807,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Repair Of Shoulder Socket Cartilage Using An E...,O,aetna,NaN,2023-03-15,gross charge,31355.71,gross
1,cpt,43235,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,"Diagnostic Examination Of Esophagus, Stomach, ...",O,aetna,NaN,2023-03-15,gross charge,13684.00,gross
2,cpt,36561,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Insertion Of Central Venous Catheter And Impla...,O,aetna,NaN,2023-03-15,gross charge,19250.00,gross
3,cpt,64492,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Injections Of Upper Or Middle Spine Facet Join...,O,aetna,NaN,2023-03-15,gross charge,20764.93,gross
4,cpt,29891,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Removal Of Bone Defect Of Shin And/Or Ankle Us...,O,aetna,NaN,2023-03-15,gross charge,60164.85,gross
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26320,cpt,93971,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Venous Doppler- Extremity Unilateral 93971,NaN,NaN,1714174,2023-03-15,discounted cash price,NaN,cash
26321,cpt,63662,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,REMOVAL OF SPINAL NEUROSTIMULATOR ELEC. PLATE/...,NaN,NaN,2580939,2023-03-15,discounted cash price,NaN,cash
26322,cpt,27497,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,FASCIOTOMY W/DEBRID.OF NOVIABLE THIGH OR KNEE ...,NaN,NaN,1480944,2023-03-15,discounted cash price,NaN,cash
26323,cpt,73630,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,XR Foot Complete Left,NaN,NaN,630548,2023-03-15,discounted cash price,NaN,cash


In [12]:
df.loc[df['payer_category'] != 'payer', 'payer_name'] = None
df

,code_type,code,name,run_id,hospital_ein,description,patient_class,payer_name,internal_code,file_last_updated,payer_orig,rate,payer_category
0,cpt,29807,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Repair Of Shoulder Socket Cartilage Using An E...,O,None,NaN,2023-03-15,gross charge,31355.71,gross
1,cpt,43235,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,"Diagnostic Examination Of Esophagus, Stomach, ...",O,None,NaN,2023-03-15,gross charge,13684.00,gross
2,cpt,36561,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Insertion Of Central Venous Catheter And Impla...,O,None,NaN,2023-03-15,gross charge,19250.00,gross
3,cpt,64492,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Injections Of Upper Or Middle Spine Facet Join...,O,None,NaN,2023-03-15,gross charge,20764.93,gross
4,cpt,29891,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Removal Of Bone Defect Of Shin And/Or Ankle Us...,O,None,NaN,2023-03-15,gross charge,60164.85,gross
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26320,cpt,93971,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,Venous Doppler- Extremity Unilateral 93971,NaN,None,1714174,2023-03-15,discounted cash price,NaN,cash
26321,cpt,63662,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,REMOVAL OF SPINAL NEUROSTIMULATOR ELEC. PLATE/...,NaN,None,2580939,2023-03-15,discounted cash price,NaN,cash
26322,cpt,27497,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,FASCIOTOMY W/DEBRID.OF NOVIABLE THIGH OR KNEE ...,NaN,None,1480944,2023-03-15,discounted cash price,NaN,cash
26323,cpt,73630,TOPS Surgical Specialty Hospital,6145a090-c343-11ed-a585-536ad537c29c,76-0246499,XR Foot Complete Left,NaN,None,630548,2023-03-15,discounted cash price,NaN,cash


In [13]:
df.loc[df['code_type'] == 'revCode', 'rev_code'] = df.loc[df['code_type'] == 'revCode']['code']
df.loc[df['code_type'] == 'revCode', 'code'] = 'na'
df.loc[df['code_type'] == 'revCode', 'code_prefix'] = 'none'
df.loc[df['code_type'] == 'revCode', 'code_type'] = None
df.loc[df['code_type'] == 'cpt', 'code_prefix'] = 'hcpcs_cpt'

In [14]:
df.loc[df['patient_class'] == "I", 'patient_class'] = "inpatient"
df.loc[df['patient_class'] == "O", 'patient_class'] = "outpatient"
df['patient_class'] = df['patient_class'].fillna('na')

In [15]:
del df['run_id']
del df['name']
df['hospital_ccn'] = ccn
df['filename'] = filename
df['url'] = url
df['modifier'] = 'na'
df['ndc'] = 'na'
df['rev_code'] = df['rev_code'].fillna('na')
df['billing_class'] = 'na'
df['plan_orig'] = 'na'
df['internal_code'] = 'na'
df.loc[df['rate'] == 'N/A', 'rate'] = None
df

,code_type,code,hospital_ein,description,patient_class,payer_name,internal_code,file_last_updated,payer_orig,rate,payer_category,rev_code,code_prefix,hospital_ccn,filename,url,modifier,ndc,billing_class,plan_orig
0,cpt,29807,76-0246499,Repair Of Shoulder Socket Cartilage Using An E...,outpatient,None,na,2023-03-15,gross charge,31355.71,gross,na,hcpcs_cpt,450774,760246499_TOPSSurgicalSpecialtyHospital_standa...,https://www.tops-hospital.com/wp-content/uploa...,na,na,na,na
1,cpt,43235,76-0246499,"Diagnostic Examination Of Esophagus, Stomach, ...",outpatient,None,na,2023-03-15,gross charge,13684.00,gross,na,hcpcs_cpt,450774,760246499_TOPSSurgicalSpecialtyHospital_standa...,https://www.tops-hospital.com/wp-content/uploa...,na,na,na,na
2,cpt,36561,76-0246499,Insertion Of Central Venous Catheter And Impla...,outpatient,None,na,2023-03-15,gross charge,19250.00,gross,na,hcpcs_cpt,450774,760246499_TOPSSurgicalSpecialtyHospital_standa...,https://www.tops-hospital.com/wp-content/uploa...,na,na,na,na
3,cpt,64492,76-0246499,Injections Of Upper Or Middle Spine Facet Join...,outpatient,None,na,2023-03-15,gross charge,20764.93,gross,na,hcpcs_cpt,450774,760246499_TOPSSurgicalSpecialtyHospital_standa...,https://www.tops-hospital.com/wp-content/uploa...,na,na,na,na
4,cpt,29891,76-0246499,Removal Of Bone Defect Of Shin And/Or Ankle Us...,outpatient,None,na,2023-03-15,gross charge,60164.85,gross,na,hcpcs_cpt,450774,760246499_TOPSSurgicalSpecialtyHospital_standa...,https://www.tops-hospital.com/wp-content/uploa...,na,na,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26320,cpt,93971,76-0246499,Venous Doppler- Extremity Unilateral 93971,na,None,na,2023-03-15,discounted cash price,NaN,cash,na,hcpcs_cpt,450774,760246499_TOPSSurgicalSpecialtyHospital_standa...,https://www.tops-hospital.com/wp-content/uploa...,na,na,na,na
26321,cpt,63662,76-0246499,REMOVAL OF SPINAL NEUROSTIMULATOR ELEC. PLATE/...,na,None,na,2023-03-15,discounted cash price,NaN,cash,na,hcpcs_cpt,450774,760246499_TOPSSurgicalSpecialtyHospital_standa...,https://www.tops-hospital.com/wp-content/uploa...,na,na,na,na
26322,cpt,27497,76-0246499,FASCIOTOMY W/DEBRID.OF NOVIABLE THIGH OR KNEE ...,na,None,na,2023-03-15,discounted cash price,NaN,cash,na,hcpcs_cpt,450774,760246499_TOPSSurgicalSpecialtyHospital_standa...,https://www.tops-hospital.com/wp-content/uploa...,na,na,na,na
26323,cpt,73630,76-0246499,XR Foot Complete Left,na,None,na,2023-03-15,discounted cash price,NaN,cash,na,hcpcs_cpt,450774,760246499_TOPSSurgicalSpecialtyHospital_standa...,https://www.tops-hospital.com/wp-content/uploa...,na,na,na,na


In [16]:
df_out = pd.DataFrame(columns=TARGET_COLUMNS)
df_out = df_out.append(df)
df_out = df_out.dropna(subset=['rate'], axis=0)
df_out

/tmp/ipykernel_35458/2120514363.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_out = df_out.append(df)


,hospital_ccn,hospital_ein,code_type,internal_code,description,code_orig,code_prefix,code,modifier,ndc,...,plan_name,plan_id,plan_type,rate,rate_method,rate_desc,filename,file_last_updated,url,permalink
0,450774,76-0246499,cpt,na,Repair Of Shoulder Socket Cartilage Using An E...,NaN,hcpcs_cpt,29807,na,na,...,NaN,NaN,NaN,31355.71,NaN,NaN,760246499_TOPSSurgicalSpecialtyHospital_standa...,2023-03-15,https://www.tops-hospital.com/wp-content/uploa...,NaN
1,450774,76-0246499,cpt,na,"Diagnostic Examination Of Esophagus, Stomach, ...",NaN,hcpcs_cpt,43235,na,na,...,NaN,NaN,NaN,13684.00,NaN,NaN,760246499_TOPSSurgicalSpecialtyHospital_standa...,2023-03-15,https://www.tops-hospital.com/wp-content/uploa...,NaN
2,450774,76-0246499,cpt,na,Insertion Of Central Venous Catheter And Impla...,NaN,hcpcs_cpt,36561,na,na,...,NaN,NaN,NaN,19250.00,NaN,NaN,760246499_TOPSSurgicalSpecialtyHospital_standa...,2023-03-15,https://www.tops-hospital.com/wp-content/uploa...,NaN
3,450774,76-0246499,cpt,na,Injections Of Upper Or Middle Spine Facet Join...,NaN,hcpcs_cpt,64492,na,na,...,NaN,NaN,NaN,20764.93,NaN,NaN,760246499_TOPSSurgicalSpecialtyHospital_standa...,2023-03-15,https://www.tops-hospital.com/wp-content/uploa...,NaN
4,450774,76-0246499,cpt,na,Removal Of Bone Defect Of Shin And/Or Ankle Us...,NaN,hcpcs_cpt,29891,na,na,...,NaN,NaN,NaN,60164.85,NaN,NaN,760246499_TOPSSurgicalSpecialtyHospital_standa...,2023-03-15,https://www.tops-hospital.com/wp-content/uploa...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22405,450774,76-0246499,cpt,na,Automated Urinalysis Test,NaN,hcpcs_cpt,81003,na,na,...,NaN,NaN,NaN,87.50,NaN,NaN,760246499_TOPSSurgicalSpecialtyHospital_standa...,2023-03-15,https://www.tops-hospital.com/wp-content/uploa...,NaN
22406,450774,76-0246499,cpt,na,"X-Ray Of Abdomen, 1 View",NaN,hcpcs_cpt,74018,na,na,...,NaN,NaN,NaN,225.40,NaN,NaN,760246499_TOPSSurgicalSpecialtyHospital_standa...,2023-03-15,https://www.tops-hospital.com/wp-content/uploa...,NaN
22407,450774,76-0246499,cpt,na,"Bacterial Culture, Any Other Source Except Uri...",NaN,hcpcs_cpt,87070,na,na,...,NaN,NaN,NaN,220.54,NaN,NaN,760246499_TOPSSurgicalSpecialtyHospital_standa...,2023-03-15,https://www.tops-hospital.com/wp-content/uploa...,NaN
22408,450774,76-0246499,cpt,na,Injections Of Anesthetic And/Or Steroid Drug I...,NaN,hcpcs_cpt,64479,na,na,...,NaN,NaN,NaN,2674.00,NaN,NaN,760246499_TOPSSurgicalSpecialtyHospital_standa...,2023-03-15,https://www.tops-hospital.com/wp-content/uploa...,NaN


In [17]:
df_out.to_csv(ccn + ".csv", index=False)